# 爬虫



In [1]:
import requests
from lxml import etree
import re,time
import pandas as pd
import csv
import numpy as np

url='http://tszb.sdcourt.gov.cn:81/findHomePageContents.action'
header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
url_list=[]#所有网页

#获取当前页面的所有网页
#通过分析，下一页不是通过URL的变化，而是携带data访问数据，所以为访问第二页数据我们，我们要携带data
def down_url():
    try:
        for n in range(1,100):
            data={'start': '{}'.format(n),
            'limit': '16',
            'contentsearch.tcase.casetype': '1',
            'contentsearch.tcase.casetype': '1',
            'contentsearch.ttrial.begindatetime': '',
            'contentsearch.ttrial.enddatetime': '',
            'contentsearch.ttrial.clerk': '',
            'contentsearch.tcase.party': '',
            'contentsearch.tcase.casecode': ''}
            res=requests.post(url,headers=header,data=data).text
            href=re.findall('<a  target="_blank" href=(.*)class=',res)
            for i in range(len(href)):
                new_url='http://tszb.sdcourt.gov.cn:81/'+href[i].replace('"','')
                data =[[new_url]]
                with open('F:/Code/python/folder/url.csv', 'a',newline='') as f:
                        w = csv.writer(f)
                        w.writerows(data) 
                        #time.sleep(0.7)
            print('成功获取第{}的页面所有网址'.format(n))
        print('全部完成-------------------')
    except:
        print('{}下载失败'.format(n))

#获取-时间，类型，地点
def get_text():
    for i in range(len(url_list)):
        res=requests.get(url_list[i],headers=header).text
        place=re.findall('<span class="boriui_wenzi">法院：</span><span class="bord" title="(.*?)"',res)[0]
        time=re.findall('<span class="boriui_wenzi">开庭时间：</span><span class="bord" title="(.*?)"',res)[0]
        thing_type=re.findall('											title="(.*?)">',res)[0]
        data=[[place,time,thing_type]]
        print('{}{}{}'.format(place,time,thing_type))
        with open('D:/Code/python/folder/thing.csv', 'a',newline='') as f:
            w = csv.writer(f)
            w.writerows(data)
        time.sleep(0.7)

#处理网址，将网址添加到列表中
def check():
    url_csv=pd.read_csv('D:/Code/python/url.csv',usecols=[0],header=None)
    url_csv=np.array(url_csv)
    for i in range(1,3168,2):
        url_list.append(url_csv[i][0])


# 图像分割————阈值化


In [3]:
import cv2
import  numpy as np
img=cv2.imread('car.jpg')

img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


#普通二值阈值化
t1,dst1=cv2.threshold(img,127,255,cv2.THRESH_BINARY)

#采用otsu的处理
t2,dst2=cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)


cv2.namedWindow('image',cv2.WINDOW_NORMAL)  #WINDOW_NORMAL使窗口大小可以调整
cv2.namedWindow("THRESH_TOZERO",cv2.WINDOW_NORMAL)
cv2.namedWindow("THRESH_TOZERO_INV",cv2.WINDOW_NORMAL)


#显示图像
cv2.imshow("image",img)
cv2.imshow("THRESH_BINARY",dst1)
cv2.imshow("THRESH_OTSU",dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()


# 基于纹理的分割


In [4]:
import cv2
import numpy as np  

img=cv2.imread('wyb.jpg')
r=cv2.selectROI("input",img,False)
print("input",r)



#roi区域
roi=img[int(r[1]):int(r[1]+r[3]),int(r[0]):int(r[0]+r[2])]
#原图mask
mask=np.zeros(img.shape[:2],np.uint8)
#矩形roi
rect=(int(r[0]),int(r[1]),int(r[2]),int(r[3])#包括前景的矩形，格式为(x,y,w,h)
bgdmodel=np.zeros((1,65),np.float64)#bg模型的临时数组
fgdmodel=np.zeros((1,65),np.float64)#fg模型的临时数组

CV2.grabCut(img,mask,rect,bgdmodel,fgdmodel,11,cv2.GC_INIT_WITH_RECT)


#提取前景和可能的前景区域
mask2=np.where((mask==2)|(mask==0),0,1).astype('uint8')
print(mask2.shape)
result=cv2.bitwise_and(roi,roi,mask=mask2)
cv2.imshow("result",result)
cv2.waitKey(0)
cv2.destroyAllWindows()




# <!-- cv2.imwrite("wyb_mask.jpg",mask) -->
cv2.imwrite("wyb_result.jpg",result)
cv2.imwrite("roi.jpg",roi)

SyntaxError: invalid syntax (Temp/ipykernel_156/2648141175.py, line 16)

In [2]:
import cv2
import numpy as np  

img = cv2.imread('cut.jpg')
r = cv2.selectROI("input", img, False)
print("input", r)

# roi区域
roi = img[int(r[1]):int(r[1] + r[3]), int(r[0]):int(r[0] + r[2])]

# 原图mask
mask = np.zeros(img.shape[:2], np.uint8)

# 矩形roi
rect = (int(r[0]), int(r[1]), int(r[2]), int(r[3]))  # Fix: Add the closing parenthesis

bgdmodel = np.zeros((1, 65), np.float64)  # bg模型的临时数组
fgdmodel = np.zeros((1, 65), np.float64)  # fg模型的临时数组

cv2.grabCut(img, mask, rect, bgdmodel, fgdmodel, 11, cv2.GC_INIT_WITH_RECT)

# 提取前景和可能的前景区域
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
print(mask2.shape)
result = cv2.bitwise_and(roi, roi, mask=mask2)
cv2.imshow("result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# cv2.imwrite("wyb_mask.jpg", mask)
cv2.imwrite("wyb_result.jpg", result)
cv2.imwrite("roi.jpg", roi)
